In [167]:
import numpy as np
import pickle
from matplotlib import pyplot as plt
from scipy.special import expit as sigmoid

In [259]:
# 输入层神经元为 784
# 隐藏层神经元为 20
# 输出层神经元为 10
input_size = 784
hidden_size = 50
output_size = 10
epochs = 10

In [262]:
Wh  = np.zeros((input_size,hidden_size))
Wo  = np.zeros((hidden_size,output_size))
#Wh = np.random.randn(input_size,hidden_size)
#Wo = np.random.randn(hidden_size,output_size)
#输出层偏置向量
Wob = np.random.randn(output_size)
#Wob = np.zeros(output_size)
#隐藏层偏置向量
Whb = np.random.randn(hidden_size)
#Whb = np.zeros(hidden_size)
learning_rate = 0.01

In [266]:
pkl_file = open("/home/qinjianbo/Downloads/mnist.pkl","rb")
train_set,valid_set,test_set = pickle.load(pkl_file,encoding="iso-8859-1")
train_X = train_set[0]
train_Y = train_set[1]
valid_X = valid_set[0]
valid_Y = valid_set[1]

train_X = np.concatenate((train_X,valid_X))
train_Y = np.concatenate((train_Y,valid_Y))

test_X = test_set[0]
test_Y = test_set[1]

In [238]:
#输出层节点误差项计算公式
#P :预测值
#T :真实值
def Output_diff(P,T):
    diff = P * (1 - P) * (T - P)
    return diff

def get_loss(P,T):
    return np.square(P - T).sum() / P.shape[0]

In [270]:
# 输入为x
for epoch in range(epochs):
    loss = 0.0
    for k in range(train_Y.shape[0]):
        x = train_X[k]
        Yreal = np.array([0.1] * 10)
        Yreal[train_Y[k]] = 0.9

        #前向传播
        H = sigmoid(np.matmul(Wh.T,x) + Whb) #隐藏层的值
        Ypredict = sigmoid(np.matmul(Wo.T,H) + Wob) #输出层的值

        #输出层偏导矩阵
        Wo_update_mat = np.zeros(Wo.shape)
        #隐藏层偏置更新向量
        Whb_update_mat = np.zeros(Whb.shape)
        #输出层偏置更新向量
        Wob_update_mat = np.zeros(Wob.shape)

        Ydiff = Output_diff(Ypredict,Yreal)
        for index in range(Ypredict.shape[0]):
            #Yindex = Ypredict[index] * (1 - Ypredict[index]) * (Yreal[index] - Ypredict[index])
            #Yindex = Output_diff(Ypredict[index],Yreal[index])
            Yindex = Ydiff[index]
            update_entry =  learning_rate * Yindex * H
            update_entry = update_entry.reshape(hidden_size)
            Wo_update_mat[:,index] = update_entry
            Wob_update_mat[index] = learning_rate * Yindex

        #Ydiff 是输出层误差项
        #Hdiff 是隐藏层误差项
        Hdiff = np.array([0] * H.shape[0] , dtype=float)
        for j in range(len(H)):
            asum = 0
            for i in range(len(Ydiff)):
                asum += Wo.T[i][j] * Ydiff[i]
            Hdiff[j] = H[j] * (1 - H[j]) * asum

        Wh_update_mat = np.zeros(Wh.shape)
        for index in range(H.shape[0]):
            Hindex = Hdiff[index]
            update_entry = learning_rate * Hindex * x
            update_entry = update_entry
            Wh_update_mat[:,index] = update_entry
            Whb_update_mat[index] = learning_rate * Hindex


        Wo += Wo_update_mat
        Wh += Wh_update_mat

        Whb +=Whb_update_mat
        Wob +=Wob_update_mat

        loss += get_loss(Ypredict,Yreal)
        if(k % 3000 == 0):
            print("[%s|%s] in %s,loss is %s"%(epoch,epochs,k,loss))
            loss = 0
    print("finish %s eopch"%(epoch))
    print("calculating accuracy")
    result_predict = np.array([-1] * valid_Y.shape[0])
    for index_test_X in range(test_X.shape[0]):
        H = sigmoid(np.matmul(Wh.T,test_X[index_test_X]) + Whb) #隐藏层的值
        Ypredict = sigmoid(np.matmul(Wo.T,H) + Wob) #输出层的值
        result_predict[index_test_X] = np.argmax(Ypredict)
    zero_vec = (result_predict - test_Y)
    zero_count = len(zero_vec[zero_vec == 0])
    print("[%s|%s] valid acc is %s"%(epoch+1,epochs,zero_count/test_Y.shape[0]))

[0|10] in 0,loss is 0.014419225729256729
[0|10] in 3000,loss is 25.517593596279116
[0|10] in 6000,loss is 23.498348427125073
[0|10] in 9000,loss is 27.1722074993285
[0|10] in 12000,loss is 24.30099895515183
[0|10] in 15000,loss is 30.003171717350337
[0|10] in 18000,loss is 27.735407918024276
[0|10] in 21000,loss is 23.852328245951558
[0|10] in 24000,loss is 25.027426528343792
[0|10] in 27000,loss is 26.233509097239445
[0|10] in 30000,loss is 26.25837154070371
[0|10] in 33000,loss is 29.125148690644355
[0|10] in 36000,loss is 23.81974983703534
[0|10] in 39000,loss is 26.346576517986676
[0|10] in 42000,loss is 26.90372575835605
[0|10] in 45000,loss is 26.31915895294582
[0|10] in 48000,loss is 27.830889609421302
[0|10] in 51000,loss is 28.15879388769365
[0|10] in 54000,loss is 26.56597214819458
[0|10] in 57000,loss is 23.300744843406918
finish 0 eopch
calculating accuracy
[1|10] valid acc is 0.9372
[1|10] in 0,loss is 0.012889389993851166
[1|10] in 3000,loss is 24.747737337270195
[1|10] i

[9|10] in 3000,loss is 19.914168136119354
[9|10] in 6000,loss is 18.269881879777085
[9|10] in 9000,loss is 21.45855991413839
[9|10] in 12000,loss is 19.150517071084188
[9|10] in 15000,loss is 23.11904886254199
[9|10] in 18000,loss is 21.45566003497097
[9|10] in 21000,loss is 18.933199702666432
[9|10] in 24000,loss is 19.277325309440496
[9|10] in 27000,loss is 20.58405047463515
[9|10] in 30000,loss is 20.53835173136963
[9|10] in 33000,loss is 22.521974530486734
[9|10] in 36000,loss is 18.37565871597344
[9|10] in 39000,loss is 20.697988453006946
[9|10] in 42000,loss is 21.129459479448524
[9|10] in 45000,loss is 20.69780395462524
[9|10] in 48000,loss is 22.25215700246822
[9|10] in 51000,loss is 22.585462319995052
[9|10] in 54000,loss is 21.013450010416737
[9|10] in 57000,loss is 18.061619487377047
finish 9 eopch
calculating accuracy
[10|10] valid acc is 0.9504


In [195]:
train_Y.shape[0]

50000

In [190]:
Ypredict.shape

(10,)

In [194]:
np.square(Ypredict - Yreal)

array([0.00054177, 0.00180969, 0.00139157, 0.00723062, 0.00992132,
       0.00995243, 0.01936547, 0.00888613, 0.20445254, 0.00648779])

In [193]:
np.argmax(Yreal)

4

In [271]:
zero_vec = np.array([-1] * test_Y.shape[0])
for index_test_X in range(test_X.shape[0]):
        H = sigmoid(np.matmul(Wh.T,valid_X[index_test_X]) + Whb) #隐藏层的值
        Ypredict = sigmoid(np.matmul(Wo.T,H) + Wob) #输出层的值
        result_predict[index_test_X] = np.argmax(Ypredict)
zero_vec = (result_predict - valid_Y)
zero_count = len(zero_vec[zero_vec == 0])
print("test_set acc is %s"%(zero_count/test_Y.shape[0]))

test_set acc is 0.9598


In [267]:
train_X.shape

(60000, 784)

In [268]:
train_Y.shape

(60000,)